curl -fsSL https://ollama.com/install.sh | sh

ollama serve &

ollama pull llama3

ollama pull nomic-embed-text

In [ ]:
!pip install langchain_community bs4 chromadb colab-xterm
%load_ext colabxterm
%xterm

In [20]:
from langchain_community.llms import Ollama
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma

ollama = Ollama(
    base_url='http://localhost:11434',
    model="llama3"
)

loader = WebBaseLoader("https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1&lang=kor")
data = loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
all_splits = text_splitter.split_documents(data)

oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=oembed)

question="경북대 IT대학 컴퓨터학부의 가장 최근 공지는 뭐야?"
docs = vectorstore.similarity_search(question)

In [ ]:
from langchain.chains import RetrievalQA
qachain=RetrievalQA.from_chain_type(ollama, retriever=vectorstore.as_retriever())
res = qachain.invoke({"query": question})
print(res['result'])

[참고](https://github.com/ollama/ollama/blob/main/docs/tutorials/langchainpy.md)